# Install Libraries


In [ ]:
!pip install pymysql

     |████████████████████████████████| 51kB 5.7MB/s 


In [ ]:
import pymysql as pm
import pandas as pd
import pymysql.cursors

## **Function to get a state's data from the COVIDNow Dataset**

In [ ]:
def getrows(state,start,end):
  conn = pm.connect(
      host = 'host',
      user = 'admin',
      password = 'password',
      db = 'dbname',
      cursorclass = pymysql.cursors.DictCursor)
  try:
      cur = conn.cursor()
      print("Connection Established")
      Cmd = '''
      SELECT `date`, county, fips, `actuals.newCases`, 
      AVG(`actuals.newCases`) OVER (PARTITION BY county order by `date` ROWS between 7 preceding and current row) as 7day_Rolling_Avg_cases,
      `actuals.newDeaths`, AVG(`actuals.newDeaths`) OVER (PARTITION BY county order by `date` ROWS between 7 preceding and current row) as 7day_Rolling_Avg_deaths,
      `metrics.testPositivityRatio`, `metrics.caseDensity`, `metrics.infectionRate`, 
      round((`actuals.newCases`/`metrics.testPositivityRatio`),0) AS `daily tests`,
      round(((`actuals.newCases`/`metrics.testPositivityRatio`)-`actuals.newCases`),0) AS `daily negatives`
      FROM covidNOW.CAN_CBAP_timeseries '''+'''WHERE state = "''' + str(state) + '''" AND (`date` BETWEEN "'''+ str(start) + '''" AND "''' + str(end) + '''") ;'''
      cur.execute(Cmd)
      rows = cur.fetchall()
      return rows
  finally:
      conn.close()

## **Function to get weather information**


In [ ]:
def getrows1(start,end):
  conn = pm.connect(
      host = 'host',
      user = 'admin',
      password = 'password',
      db = 'dbname',
      cursorclass = pymysql.cursors.DictCursor)
  try:
      cur = conn.cursor()
      print("Connection Established")
      Cmd = '''
      SELECT county_fips_code AS fips, `date`, avg_temperature_air_2m_f AS temperature, avg_humidity_relative_2m_pct AS humidity
      FROM covidNOW.demo_temp '''+'''WHERE (`date` BETWEEN "'''+ str(start) + '''" AND "''' + str(end) + '''") ;'''
      cur.execute(Cmd)
      rows = cur.fetchall()
      return rows
  finally:
      conn.close()

## **Function to get demographic information for a state**




In [ ]:
def getstatic(state):
  conn = pm.connect(
      host = 'host',
      user = 'admin',
      password = 'password',
      db = 'dbname',
      cursorclass = pymysql.cursors.DictCursor)
  try:
      cur = conn.cursor()
      print("Connection Established")
      Cmd = "SELECT state, fips, lat, lon, population, pop_density, diabetes, heart_disease, sex_ratio, median_age, white_pop, asian_pop, black_pop, hispanic_pop FROM covidNOW.static WHERE state = '" + str(state) + "';"
      cur.execute(Cmd)
      rows = cur.fetchall()
      return rows
  finally:
      conn.close()

# **Enter State**



In [ ]:
state = 'NC'

In [ ]:
#Provide start and end date
start_date = pd.to_datetime('01-01-2020')
end_date = pd.to_datetime('05-16-2021')

# **The covidnow dataset**

In [ ]:
covidnow = getrows(state, start_date, end_date)
df = pd.DataFrame(covidnow)
#Make sure 'date' column is interpreted as a date and not a string
df['date'] = pd.to_datetime(df['date'])

#Change FIPS code from string to integer type 
df['fips'] = df['fips'].astype(int)
df.head(-10)

Connection Established


,date,county,fips,actuals.newCases,7day_Rolling_Avg_cases,actuals.newDeaths,7day_Rolling_Avg_deaths,metrics.testPositivityRatio,metrics.caseDensity,metrics.infectionRate,daily tests,daily negatives
0,2020-01-22,Alamance County,37001,NaN,NaN,NaN,NaN,0.0000,0.000000,NaN,NaN,NaN
1,2020-01-23,Alamance County,37001,NaN,NaN,NaN,NaN,0.0000,0.000000,NaN,NaN,NaN
2,2020-01-24,Alamance County,37001,NaN,NaN,NaN,NaN,0.0000,0.000000,NaN,NaN,NaN
3,2020-01-25,Alamance County,37001,NaN,NaN,NaN,NaN,0.0000,0.000000,NaN,NaN,NaN
4,2020-01-26,Alamance County,37001,NaN,NaN,NaN,NaN,0.0000,0.000000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
48085,2021-05-02,Yancey County,37199,0.0,1.857143,0.0,0.0,0.1288,11.991071,0.905950,0.0,0.0
48086,2021-05-03,Yancey County,37199,8.0,3.000000,0.0,0.0,0.1269,16.603022,0.895553,63.0,55.0
48087,2021-05-04,Yancey County,37199,0.0,2.625000,0.0,0.0,0.1475,15.021782,0.883938,0.0,0.0
48088,2021-05-05,Yancey County,37199,9.0,3.500000,0.0,0.0,0.1500,18.974882,0.871150,60.0,51.0


In [ ]:
df[df['county'] == 'Wake County']

,date,county,fips,actuals.newCases,7day_Rolling_Avg_cases,actuals.newDeaths,7day_Rolling_Avg_deaths,metrics.testPositivityRatio,metrics.caseDensity,metrics.infectionRate,daily tests,daily negatives
43771,2020-01-22,Wake County,37183,NaN,NaN,NaN,NaN,0.0000,0.000000,NaN,NaN,NaN
43772,2020-01-23,Wake County,37183,NaN,NaN,NaN,NaN,0.0000,0.000000,NaN,NaN,NaN
43773,2020-01-24,Wake County,37183,NaN,NaN,NaN,NaN,0.0000,0.000000,NaN,NaN,NaN
43774,2020-01-25,Wake County,37183,NaN,NaN,NaN,NaN,0.0000,0.000000,NaN,NaN,NaN
43775,2020-01-26,Wake County,37183,NaN,NaN,NaN,NaN,0.0000,0.000000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
44247,2021-05-12,Wake County,37183,147.0,135.00,0.0,0.285714,0.0466,12.168597,0.856199,3155.0,3008.0
44248,2021-05-13,Wake County,37183,113.0,132.50,3.0,0.571429,0.0452,10.665191,0.850852,2500.0,2387.0
44249,2021-05-14,Wake County,37183,0.0,103.75,0.0,0.428571,0.0425,9.020438,0.846148,0.0,0.0
44250,2021-05-15,Wake County,37183,218.0,115.00,0.0,0.428571,0.0414,10.048408,0.842155,5266.0,5048.0


## **Weather time series dataset**

In [ ]:
weather_data = getrows1(start_date, end_date)
df2 = pd.DataFrame(weather_data)

# Change 'date' to datetime datatype 
df2['date'] = pd.to_datetime(df2['date'])
# Change FIPS code to INT64 from FLOAT64
df2['fips'] = df2['fips'].astype(int)
df2.head()

Connection Established


,fips,date,temperature,humidity
0,17063,2021-01-18,25.7,86.0
1,8061,2021-01-18,34.3,54.0
2,8083,2021-01-18,33.9,54.0
3,19181,2021-01-18,25.3,84.0
4,46025,2021-01-18,22.8,92.0


# **Static demographics dataset**

In [ ]:
static = getstatic(state)
df3 = pd.DataFrame(static)

# Change FIPS code and all population numbers to INT64 from FLOAT64
df3[['fips','population','white_pop','asian_pop','black_pop','hispanic_pop']] = df3[['fips','population','white_pop','asian_pop','black_pop','hispanic_pop']].astype(int)
df3.head(10)

Connection Established


,state,fips,lat,lon,population,pop_density,diabetes,heart_disease,sex_ratio,median_age,white_pop,asian_pop,black_pop,hispanic_pop
0,NC,37001,36.037343,-79.398844,169509,146.415457,11.53,317.2,90.443208,39.5,103713,2478,30569,19905
1,NC,37003,35.916776,-81.172017,37497,10.714032,10.53,297.2,104.208615,43.4,32245,402,2105,1723
2,NC,37005,36.491055,-81.126195,11137,18.073270,10.37,314.9,96.402363,49.5,9548,26,255,1054
3,NC,37007,34.976201,-80.104771,24446,18.384600,13.76,439.3,110.514932,40.5,11288,335,12333,999
4,NC,37009,36.433254,-81.498915,27203,24.330337,9.91,304.0,96.695550,47.7,24800,114,199,1336
5,NC,37011,36.074592,-81.919816,17557,27.319609,10.59,334.7,118.407588,44.4,15298,149,770,896
6,NC,37013,35.494190,-76.869659,46994,21.904449,10.90,349.4,92.561343,45.7,31094,38,11826,3609
7,NC,37015,36.065336,-76.983187,18947,10.847893,14.84,347.7,101.497590,45.2,6735,125,11899,408
8,NC,37017,34.608631,-78.557235,32722,14.904328,13.20,489.5,90.577748,43.6,18466,35,11648,2523
9,NC,37019,34.074922,-78.252254,142820,57.619547,10.02,311.4,93.446073,52.8,103713,779,12966,6028


## **TOPSIS input creator**

## **Enter Date Range**

In [ ]:
#Provide start and end date
start_date = pd.to_datetime('07-30-2020')
end_date = pd.to_datetime('12-28-2020')

In [ ]:
t1 = df.loc[df.date.between(start_date, end_date+ pd.to_timedelta(6, unit = 'D')),
               ['county','fips','metrics.testPositivityRatio']].groupby(['county'],as_index = False).mean()

t21 = df.loc[df.date.between(start_date-pd.to_timedelta(7,unit ='D'), start_date-pd.to_timedelta(1, unit = 'D')),
               ['county','actuals.newCases']].groupby(['county'], as_index = False).sum()

t22 = df.loc[df.date.between(end_date, end_date+pd.to_timedelta(6, unit = 'D')),
               ['county','actuals.newCases']].groupby(['county'], as_index = False).sum()

t1['4weekAvgcases'] = (t22['actuals.newCases'] - t21['actuals.newCases'])/4

In [ ]:
t1 = t1.merge(df3, on = 'fips', how = 'inner')
t1['4weekAvgcases'] = t1['4weekAvgcases']/(t1['population']/100000)

In [ ]:
t1.iloc[:,[0,1,2,3,5,6,7]]

,county,fips,metrics.testPositivityRatio,4weekAvgcases,lat,lon,population
0,Alamance County,37001,0.081743,83.623878,36.037343,-79.398844,169509
1,Alexander County,37003,0.136213,174.013921,35.916776,-81.172017,37497
2,Alleghany County,37005,0.082939,116.728024,36.491055,-81.126195,11137
3,Anson County,37007,0.088119,84.880962,34.976201,-80.104771,24446
4,Ashe County,37009,0.066403,99.253759,36.433254,-81.498915,27203
...,...,...,...,...,...,...,...
95,Wayne County,37191,0.077348,69.032169,35.354641,-78.008452,123131
96,Wilkes County,37193,0.089969,116.207683,36.201538,-81.161330,68412
97,Wilson County,37195,0.092860,77.016173,35.698770,-77.920576,81801
98,Yadkin County,37197,0.114218,118.804258,36.155268,-80.663555,37667


## **Master database creator**

In [ ]:
inter_df = df.merge(df2, how = 'inner', on = ['date','fips'])

In [ ]:
master_df = inter_df.merge(df3, how = 'inner', on = 'fips')
master_df.head(-10)

,date,county,fips,actuals.newCases,7day_Rolling_Avg_cases,actuals.newDeaths,7day_Rolling_Avg_deaths,metrics.testPositivityRatio,metrics.caseDensity,metrics.infectionRate,daily tests,daily negatives,temperature,humidity,state,lat,lon,population,pop_density,diabetes,heart_disease,sex_ratio,median_age,white_pop,asian_pop,black_pop,hispanic_pop
0,2020-05-01,Alamance County,37001,7.0,5.375,0.0,0.125,0.0640,2.781142,1.048076,109.0,102.0,56.4,73.0,NC,36.037343,-79.398844,169509,146.415457,11.53,317.2,90.443208,39.5,103713,2478,30569,19905
1,2020-05-01,Alamance County,37001,7.0,5.375,0.0,0.125,0.0640,2.781142,1.048076,109.0,102.0,56.4,73.0,NC,36.037343,-79.398844,169509,146.415457,11.53,317.2,90.443208,39.5,103713,2478,30569,19905
2,2020-05-01,Alamance County,37001,7.0,5.375,0.0,0.125,0.0640,2.781142,1.048076,109.0,102.0,56.4,73.0,NC,36.037343,-79.398844,169509,146.415457,11.53,317.2,90.443208,39.5,103713,2478,30569,19905
3,2020-05-02,Alamance County,37001,15.0,6.000,1.0,0.125,0.0504,3.708189,1.030592,298.0,283.0,59.9,66.0,NC,36.037343,-79.398844,169509,146.415457,11.53,317.2,90.443208,39.5,103713,2478,30569,19905
4,2020-05-02,Alamance County,37001,15.0,6.000,1.0,0.125,0.0504,3.708189,1.030592,298.0,283.0,59.9,66.0,NC,36.037343,-79.398844,169509,146.415457,11.53,317.2,90.443208,39.5,103713,2478,30569,19905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113985,2021-05-12,Yancey County,37199,3.0,3.250,0.0,0.000,0.0625,13.440541,0.767932,48.0,45.0,46.3,86.0,NC,35.894626,-82.309960,18069,21.821617,10.64,294.3,95.886462,47.4,16360,43,129,899
113986,2021-05-12,Yancey County,37199,3.0,3.250,0.0,0.000,0.0625,13.440541,0.767932,48.0,45.0,46.3,86.0,NC,35.894626,-82.309960,18069,21.821617,10.64,294.3,95.886462,47.4,16360,43,129,899
113987,2021-05-12,Yancey County,37199,3.0,3.250,0.0,0.000,0.0625,13.440541,0.767932,48.0,45.0,46.3,86.0,NC,35.894626,-82.309960,18069,21.821617,10.64,294.3,95.886462,47.4,16360,43,129,899
113988,2021-05-13,Yancey County,37199,0.0,2.125,0.0,0.000,0.0667,10.278061,0.754351,0.0,0.0,47.9,69.0,NC,35.894626,-82.309960,18069,21.821617,10.64,294.3,95.886462,47.4,16360,43,129,899


In [ ]:
from google.colab import files
master_df.to_csv('data.csv')
files.download('data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>